# House Prices: Advanced Regression Techniques

<p><a name="sections"></a></p>


## Sections

- <a href="#description">Description</a><br>
- <a href="#executive_summary">Executive Summary</a><br>
- <a href="#data_import">Data Import</a><br>
- <a href="#Feature Engineering (Advanced)">Feature Engineering (Advanced)</a><br>
- <a href="#data_cleaning_and_eda">Data Cleaning and EDA</a><br>
- <a href="#models">Models</a><br>
    - <a href="#lasso">Lasso</a><br>
    - <a href="#decision_tree">Decision Tree</a><br>
    - <a href="#bagging">Bagging</a><br>
    - <a href="#random_forest">Random Forest</a><br>
    - <a href="#boosting">Boosting</a><br>
    - <a href="#xgboost">XGBoost</a><br>
- <a href="#submission">Submission</a><br>

<p><a name="description"></a></p>

## Description

<p><a name="executive_summary"></a></p>

## Executive Summary

<p><a name="data_import_and_cleaning"></a></p>

## Data Import and Cleaning

**import libaries**

In [630]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from scipy.stats import skew
from sklearn.model_selection import cross_val_score, train_test_split,GridSearchCV
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV,ElasticNet, Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, make_scorer, mean_squared_log_error
from sklearn.model_selection import RepeatedKFold
from sklearn.neighbors import KNeighborsRegressor
import xgboost as xgb
# from sklearn.model_selection import RepeatedKFold #why it doesn't work???
# from feature_selection import VarianceReduction

In [420]:
import sys
print(sys.version)

3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


Note: ImportError: cannot import name 'RepeatedKFold'

Solution (Windows):

conda config --append channels conda-forge

conda install scikit-learn=0.19.2

**read training data**

In [574]:
train = pd.read_csv('train.csv')
print("train : " + str(train.shape))
train = train.reindex(columns=sorted(train.columns))
train.head(1)

train : (1460, 81)


,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BldgType,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,...,SaleType,ScreenPorch,Street,TotRmsAbvGrd,TotalBsmtSF,Utilities,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold
0,856,854,0,NaN,3,1Fam,TA,No,706,0,...,WD,0,Pave,8,856,AllPub,0,2003,2003,2008


In [575]:
test = pd.read_csv('test.csv')
print("test : " + str(test.shape))
test = test.reindex(columns=sorted(train.columns))
test.head(1)

test : (1459, 80)


,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BldgType,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,...,SaleType,ScreenPorch,Street,TotRmsAbvGrd,TotalBsmtSF,Utilities,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold
0,896,0,0,NaN,2,1Fam,TA,No,468.0,144.0,...,WD,120,Pave,5,882.0,AllPub,140,1961,1961,2010


In [576]:
# Check for duplicates 
idsUnique = len(set(train.Id))
idsTotal = train.shape[0]
idsDupli = idsTotal - idsUnique
print("There are " + str(idsDupli) + " duplicate IDs for " + str(idsTotal) + " total entries")
target_var=train[['SalePrice']]

There are 0 duplicate IDs for 1460 total entries


In [577]:
# combine train and test sets
data = pd.concat([train,test],axis=0)
data.reset_index(inplace=True,drop=True)
print(data.loc[[0,2000],'Id'], data.shape)

0          1
2000    2001
Name: Id, dtype: int64 (2919, 81)


In [578]:
train=data.drop(['SalePrice'],axis=1)

## Data Types

In [579]:
#data types
# numeric data
train_numeric = train._get_numeric_data()
# all columns
cols_all = train.columns.tolist()
# numeric columns
cols_numeric = train_numeric.columns.tolist()
# categoric columns
cols_categoric = list(set(cols_all)-set(cols_numeric))
# categoric data
train_categoric = train.loc[:,cols_categoric]
print('number of numeric columns: {}'.format(len(cols_numeric)))
print('number categoric columns: {}'.format(len(cols_categoric)))


number of numeric columns: 37
number categoric columns: 43


In [580]:
# Differentiate numerical features (minus the target) and categorical features
categorical_features = train.select_dtypes(include = ["object"]).columns
numerical_features = train.select_dtypes(exclude = ["object"]).columns
print("Numerical features : " + str(len(numerical_features)))
print("Categorical features : " + str(len(categorical_features)))
train_num = train[numerical_features]
train_cat = train[categorical_features]

Numerical features : 37
Categorical features : 43


In [581]:
df=train_num
print("Numerical features w/ NAs: " +str(df.columns[df.isnull().any()].values))
df=train_cat
print("Categorical features w/ NAs: " +str(df.columns[df.isnull().any()].values))

Numerical features w/ NAs: ['BsmtFinSF1' 'BsmtFinSF2' 'BsmtFullBath' 'BsmtHalfBath' 'BsmtUnfSF'
 'GarageArea' 'GarageCars' 'GarageYrBlt' 'LotFrontage' 'MasVnrArea'
 'TotalBsmtSF']
Categorical features w/ NAs: ['Alley' 'BsmtCond' 'BsmtExposure' 'BsmtFinType1' 'BsmtFinType2'
 'BsmtQual' 'Electrical' 'Exterior1st' 'Exterior2nd' 'Fence' 'FireplaceQu'
 'Functional' 'GarageCond' 'GarageFinish' 'GarageQual' 'GarageType'
 'KitchenQual' 'MSZoning' 'MasVnrType' 'MiscFeature' 'PoolQC' 'SaleType'
 'Utilities']


## Missing value imputation

In [582]:
train['Exterior1st'].value_counts()

VinylSd    1025
MetalSd     450
HdBoard     442
Wd Sdng     411
Plywood     221
CemntBd     126
BrkFace      87
WdShing      56
AsbShng      44
Stucco       43
BrkComm       6
Stone         2
AsphShn       2
CBlock        2
ImStucc       1
Name: Exterior1st, dtype: int64

In [583]:
# Handle missing values for features where median/mean or most common value doesn't make sense
# some of the following are redundant
# Alley : data description says NA means "no alley access"
train.loc[:, "Alley"] = train.loc[:, "Alley"].fillna("None")

# BsmtQual etc : data description says NA for basement features is "no basement"
train.loc[:, "BsmtCond"] = train.loc[:, "BsmtCond"].fillna("No")
train.loc[:, "BsmtExposure"] = train.loc[:, "BsmtExposure"].fillna("No")
train.loc[:, "BsmtFinType1"] = train.loc[:, "BsmtFinType1"].fillna("No")
train.loc[:, "BsmtFinType2"] = train.loc[:, "BsmtFinType2"].fillna("No")
train.loc[:, "BsmtQual"] = train.loc[:, "BsmtQual"].fillna("No")

# NA Bsmt means 0
for col in ['BsmtFinSF1','BsmtFinSF2','TotalBsmtSF','BsmtUnfSF','BsmtHalfBath','BsmtFullBath']:
    train[col].fillna(0,inplace=True)   
    
# EnclosedPorch : NA most likely means no enclosed porch
train.loc[:, "EnclosedPorch"] = train.loc[:, "EnclosedPorch"].fillna(0)
# Fence : data description says NA means "no fence"
train.loc[:, "Fence"] = train.loc[:, "Fence"].fillna("No")
# FireplaceQu : data description says NA means "no fireplace"
train.loc[:, "FireplaceQu"] = train.loc[:, "FireplaceQu"].fillna("No")
train.loc[:, "Fireplaces"] = train.loc[:, "Fireplaces"].fillna(0)
# Functional : data description says NA means typical
train.loc[:, "Functional"] = train.loc[:, "Functional"].fillna("Typ")
# GarageType etc : data description says NA for garage features is "no garage"
train.loc[:, "GarageType"] = train.loc[:, "GarageType"].fillna("No")
train.loc[:, "GarageFinish"] = train.loc[:, "GarageFinish"].fillna("No")
train.loc[:, "GarageQual"] = train.loc[:, "GarageQual"].fillna("No")
train.loc[:, "GarageCond"] = train.loc[:, "GarageCond"].fillna("No")
train.loc[:, "GarageArea"] = train.loc[:, "GarageArea"].fillna(0)
train.loc[:, "GarageCars"] = train.loc[:, "GarageCars"].fillna(0)
# HalfBath : NA most likely means no half baths above grade
train.loc[:, "HalfBath"] = train.loc[:, "HalfBath"].fillna(0)
# HeatingQC : NA most likely means typical
train.loc[:, "HeatingQC"] = train.loc[:, "HeatingQC"].fillna("TA")
# KitchenAbvGr : NA most likely means 0
train.loc[:, "KitchenAbvGr"] = train.loc[:, "KitchenAbvGr"].fillna(0)
# KitchenQual : NA most likely means typical
train.loc[:, "KitchenQual"] = train.loc[:, "KitchenQual"].fillna("TA")
# LotFrontage : NA most likely means no lot frontage
train.loc[:, "LotFrontage"] = train.loc[:, "LotFrontage"].fillna(0)
# LotShape : NA most likely means regular
train.loc[:, "LotShape"] = train.loc[:, "LotShape"].fillna("Reg")
# MasVnrType : NA most likely means no veneer
train.loc[:, "MasVnrType"] = train.loc[:, "MasVnrType"].fillna("None")
train.loc[:, "MasVnrArea"] = train.loc[:, "MasVnrArea"].fillna(0)
# MiscFeature : data description says NA means "no misc feature"
train.loc[:, "MiscFeature"] = train.loc[:, "MiscFeature"].fillna("No")
train.loc[:, "MiscVal"] = train.loc[:, "MiscVal"].fillna(0)
# OpenPorchSF : NA most likely means no open porch
train.loc[:, "OpenPorchSF"] = train.loc[:, "OpenPorchSF"].fillna(0)
# PavedDrive : NA most likely means not paved
train.loc[:, "PavedDrive"] = train.loc[:, "PavedDrive"].fillna("N")
# PoolQC : data description says NA means "no pool"
train.loc[:, "PoolQC"] = train.loc[:, "PoolQC"].fillna("No")
train.loc[:, "PoolArea"] = train.loc[:, "PoolArea"].fillna(0)
# SaleCondition : NA most likely means normal sale
train.loc[:, "SaleCondition"] = train.loc[:, "SaleCondition"].fillna("Normal")
# ScreenPorch : NA most likely means no screen porch
train.loc[:, "ScreenPorch"] = train.loc[:, "ScreenPorch"].fillna(0)
# TotRmsAbvGrd : NA most likely means 0
train.loc[:, "TotRmsAbvGrd"] = train.loc[:, "TotRmsAbvGrd"].fillna(0)
# Utilities : NA most likely means all public utilities
train.loc[:, "Utilities"] = train.loc[:, "Utilities"].fillna("AllPub")
# Electrical : NA most likely means SBrkr
train.loc[:, "Electrical"] = train.loc[:, "Electrical"].fillna("SBrkr")
# GarageYrBlt: NA same as 
train.loc[train.GarageYrBlt.isnull(), "GarageYrBlt"] = train.loc[train.GarageYrBlt.isnull(), "YearBuilt"]

# fill w/ mode
for col in ['Exterior1st','Exterior2nd','SaleType']:
    train[col].fillna(data[col].mode()[0],inplace=True)    
train['MSZoning'] =  train.groupby('Neighborhood')['MSZoning'].transform(lambda x: x.fillna(x.mode()[0]))

In [584]:
df=train
print("Features w/ NAs: " +str(df.columns[df.isnull().any()].values))

Features w/ NAs: []


In [585]:
# Some numerical features are actually really categories
train = train.replace({"MSSubClass" : {20 : "SC20", 30 : "SC30", 40 : "SC40", 45 : "SC45", 
                                       50 : "SC50", 60 : "SC60", 70 : "SC70", 75 : "SC75", 
                                       80 : "SC80", 85 : "SC85", 90 : "SC90", 120 : "SC120", 
                                       150 : "SC150", 160 : "SC160", 180 : "SC180", 190 : "SC190"},
                       "MoSold" : {1 : "Jan", 2 : "Feb", 3 : "Mar", 4 : "Apr", 5 : "May", 6 : "Jun",
                                   7 : "Jul", 8 : "Aug", 9 : "Sep", 10 : "Oct", 11 : "Nov", 12 : "Dec"}
                      })

In [586]:
# non-missing dataset
categorical_features = train.select_dtypes(include = ["object"]).columns
numerical_features = train.select_dtypes(exclude = ["object"]).columns
print("Numerical features : " + str(len(numerical_features)))
print("Categorical features : " + str(len(categorical_features)))
train_num = train[numerical_features]
train_cat = train[categorical_features]
print("NAs for numerical features in train : " + str(train_num.isnull().values.sum()))
train_num = train_num.fillna(train_num.median())
print("Remaining NAs for numerical features in train : " + str(train_num.isnull().values.sum()))

Numerical features : 35
Categorical features : 45
NAs for numerical features in train : 0
Remaining NAs for numerical features in train : 0


## Basic final dataset output

In [587]:
# generate dummy variables for categorical variables and numeric variables=0;
train_cat_dum = pd.get_dummies(train_cat)
cols=train_num.columns[train_num.apply(lambda x: x==0).sum()>0]
train_num_absent = train_num.loc[:,cols].apply(lambda x: x==0).astype(int).add_suffix('_absent')
print("Number of numerical variables: {}".format(len(train_num.columns)))
print("Number of categorical dummies: {}".format(len(train_cat_dum.columns)))
print("Number of dummies for 0s: {}".format(len(train_num_absent.columns)))
# Join categorical and numerical features
pd.concat([target_var,train_num, train_cat_dum,train_num_absent], axis = 1).to_csv('final_dataset.csv',index=False)
# data clean is done. Wrap the above into a .py file

Number of numerical variables: 35
Number of categorical dummies: 293
Number of dummies for 0s: 24


## Feature Engineering (Naive)
#### see a separate file for initial exploration. The following transform is built on many trials and previous Kaggle blogs.

## Feature Engineering (Advanced)

In [588]:
train = train.replace({"Alley" : {"Grvl" : 1, "Pave" : 2},
                       "BsmtCond" : {"No" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "BsmtExposure" : {"No" : 0, "Mn" : 1, "Av": 2, "Gd" : 3},
                       "BsmtFinType1" : {"No" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4, 
                                         "ALQ" : 5, "GLQ" : 6},
                       "BsmtFinType2" : {"No" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4, 
                                         "ALQ" : 5, "GLQ" : 6},
                       "BsmtQual" : {"No" : 0, "Po" : 1, "Fa" : 2, "TA": 3, "Gd" : 4, "Ex" : 5},
                       "ExterCond" : {"Po" : 1, "Fa" : 2, "TA": 3, "Gd": 4, "Ex" : 5},
                       "ExterQual" : {"Po" : 1, "Fa" : 2, "TA": 3, "Gd": 4, "Ex" : 5},
                       "FireplaceQu" : {"No" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "Functional" : {"Sal" : 1, "Sev" : 2, "Maj2" : 3, "Maj1" : 4, "Mod": 5, 
                                       "Min2" : 6, "Min1" : 7, "Typ" : 8},
                       "GarageCond" : {"No" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "GarageQual" : {"No" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "HeatingQC" : {"Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "KitchenQual" : {"Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "LandSlope" : {"Sev" : 1, "Mod" : 2, "Gtl" : 3},
                       "LotShape" : {"IR3" : 1, "IR2" : 2, "IR1" : 3, "Reg" : 4},
                       "PavedDrive" : {"N" : 0, "P" : 1, "Y" : 2},
                       "PoolQC" : {"No" : 0, "Fa" : 1, "TA" : 2, "Gd" : 3, "Ex" : 4},
                       "Street" : {"Grvl" : 1, "Pave" : 2},
                       "Utilities" : {"ELO" : 1, "NoSeWa" : 2, "NoSewr" : 3, "AllPub" : 4}}
                     )


In [589]:
# Create new features
# 1* Simplifications of existing features
train.OverallQual.replace({1 : 1, 2 : 1, 3 : 1, # bad
                   4 : 2, 5 : 2, 6 : 2, # average
                   7 : 3, 8 : 3, 9 : 3, 10 : 3 # good
                  }, inplace=True)
train.OverallCond.replace({1 : 1, 2 : 1, 3 : 1, # bad
                   4 : 2, 5 : 2, 6 : 2, # average
                   7 : 3, 8 : 3, 9 : 3, 10 : 3 # good
                  }, inplace=True)
train.PoolQC.replace({1 : 1, 2 : 1, # average
         3 : 2, 4 : 2 # good
        }, inplace=True)
train.GarageCond.replace({1 : 1, # bad
                 2 : 1, 3 : 1, # average
                 4 : 2, 5 : 2 # good
                }, inplace=True)
train.GarageQual.replace({1 : 1, # bad
                 2 : 1, 3 : 1, # average
                 4 : 2, 5 : 2 # good
                }, inplace=True)
train.FireplaceQu.replace({1 : 1, # bad
                   2 : 1, 3 : 1, # average
                   4 : 2, 5 : 2 # good
                  }, inplace=True)
train.FireplaceQu.replace({1 : 1, # bad
                   2 : 1, 3 : 1, # average
                   4 : 2, 5 : 2 # good
                  }, inplace=True)
train.Functional.replace({1 : 1, 2 : 1, # bad
                 3 : 2, 4 : 2, # major
                 5 : 3, 6 : 3, 7 : 3, # minor
                 8 : 4 # typical
                }, inplace=True)
train.KitchenQual.replace({1 : 1, # bad
                   2 : 1, 3 : 1, # average
                   4 : 2, 5 : 2 # good
                  }, inplace=True)
train.HeatingQC.replace({1 : 1, # bad
               2 : 1, 3 : 1, # average
               4 : 2, 5 : 2 # good
              }, inplace=True)
train.BsmtFinType1.replace({1 : 1, # unfinished
                     2 : 1, 3 : 1, # rec room
                     4 : 2, 5 : 2, 6 : 2 # living quarters
                    }, inplace=True)
train.BsmtFinType2.replace({1 : 1, # unfinished
                     2 : 1, 3 : 1, # rec room
                     4 : 2, 5 : 2, 6 : 2 # living quarters
                    }, inplace=True)
train.BsmtCond.replace({1 : 1, # bad
             2 : 1, 3 : 1, # average
             4 : 2, 5 : 2 # good
            }, inplace=True)
train.BsmtQual.replace({1 : 1, # bad
             2 : 1, 3 : 1, # average
             4 : 2, 5 : 2 # good
            }, inplace=True)
train.ExterCond.replace({1 : 1, # bad
               2 : 1, 3 : 1, # average
               4 : 2, 5 : 2 # good
              }, inplace=True)
train.ExterQual.replace({1 : 1, # bad
               2 : 1, 3 : 1, # average
               4 : 2, 5 : 2 # good
              }, inplace=True)

# Has masonry veneer or not
train.MasVnrType.replace({"BrkCmn" : 1, "BrkFace" : 1, "CBlock" : 1, 
                                               "Stone" : 1, "None" : 0}, inplace=True)


# House completed before sale or not
train.SaleCondition.replace({"Abnorml" : 0, "Alloca" : 0, "AdjLand" : 0, 
                                                      "Family" : 0, "Normal" : 0, "Partial" : 1}, inplace=True)

In [590]:
train["Fireplaces"].value_counts()

0    1420
1    1268
2     219
3      11
4       1
Name: Fireplaces, dtype: int64

In [591]:
def comb_cols_add(col1,col2):
    train[col1]=train[col1]/train[col1].max() + train[col2]/train[col2].max()
    return (train.drop(col2,axis=1))
def comb_cols_mul(col1,col2):
    train[col2]=train[col1]*train[col2]
    return (train)

In [592]:
# 2* Combinations of existing features
# Overall quality of the house
train=comb_cols_add("OverallQual","OverallCond") #normalize ratings
train=comb_cols_add("ExterQual","ExterCond")
train=comb_cols_add("GarageQual","GarageCond")
train.drop(["FireplaceQu"], inplace=True, axis=1) #litte difference from Fireplaces
train['PoolQC']=train['PoolQC']*train["PoolArea"]
train.drop(["PoolArea"], inplace=True, axis=1) #litte difference from Pool Quality
# Combine area variables
train["BsmtTotalBath"] = train["BsmtFullBath"] + (0.5 * train["BsmtHalfBath"]) 
train["TotalBath"]=train["FullBath"] + (0.5 * train["HalfBath"])
train['BsmtFinSF']=train.BsmtFinSF1+train.BsmtFinSF2
train.drop(['BsmtFullBath',"BsmtHalfBath","FullBath","HalfBath","TotalBsmtSF","1stFlrSF","2ndFlrSF",'BsmtFinSF2','BsmtFinSF1'], axis=1, inplace=True)

# Total SF for porch
train["AllPorchSF"] = train["OpenPorchSF"] + train["EnclosedPorch"] + \
train["3SsnPorch"] + train["ScreenPorch"]
train.drop(['OpenPorchSF',"EnclosedPorch","3SsnPorch","ScreenPorch"], axis=1, inplace=True)

In [593]:
# Transform Time variables
for s in ['Year','Yr']:
    cols = [col for col in train_num.columns if s in col]
    print(cols)
    for col in cols:
        train[col]=train.YrSold-train[col]
train.drop(['YrSold'],axis=1, inplace=True)

['YearBuilt', 'YearRemodAdd']
['GarageYrBlt', 'YrSold']


In [594]:
categorical_features = train.select_dtypes(include = ["object"]).columns
numerical_features = train.select_dtypes(exclude = ["object"]).columns
print("Numerical features : " + str(len(numerical_features)))
print("Categorical features : " + str(len(categorical_features)))
train_num = train[numerical_features]
train_cat = train[categorical_features]
print("NAs for numerical features in train : " + str(train_num.isnull().values.sum()))
train_num = train_num.fillna(train_num.median())
print("Remaining NAs for numerical features in train : " + str(train_num.isnull().values.sum()))

Numerical features : 41
Categorical features : 24
NAs for numerical features in train : 0
Remaining NAs for numerical features in train : 0


In [595]:
train.groupby(["MSSubClass","BldgType"]).size() #wait for peter to code up the hierachical grouping for MSSub

MSSubClass  BldgType
SC120       1Fam           5
            Twnhs         15
            TwnhsE       162
SC150       Twnhs          1
SC160       1Fam           1
            Twnhs         71
            TwnhsE        56
SC180       Twnhs          9
            TwnhsE         8
SC190       1Fam           1
            2fmCon        60
SC20        1Fam        1077
            2fmCon         1
            TwnhsE         1
SC30        1Fam         139
SC40        1Fam           6
SC45        1Fam          18
SC50        1Fam         287
SC60        1Fam         574
            2fmCon         1
SC70        1Fam         128
SC75        1Fam          23
SC80        1Fam         118
SC85        1Fam          48
SC90        Duplex       109
dtype: int64

## Dummy variables

In [596]:
# generate dummy variables for categorical variables and numeric variables=0;
train_cat.drop(['Exterior2nd','MSSubClass'],axis=1,inplace=True) #Exterior2nd and Exterior1st highly correlated
train_cat_dum = pd.get_dummies(train_cat)

/Users/zlu15/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [597]:
# NA dummies analysis
def zero_means_no(s):
    cols = [col for col in train_num.columns if s in col]
    df=train_num[cols]==0 #akward
    if any(df.any())>0:
        print('num vars',cols)
        cols = [col for col in train_cat.columns if s in col]
        print('cat vars',cols)
        cols = [col for col in train_cat_dum.columns if s in col]
        print('cat var dummies,',cols)
for s in ['Bsmt','Mas','Lot','Pool','Garage','Porch','Deck','Kitchen','Exter']:
    zero_means_no(s)

num vars ['BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'BsmtQual', 'BsmtUnfSF', 'BsmtTotalBath', 'BsmtFinSF']
cat vars []
cat var dummies, []
num vars ['MasVnrArea', 'MasVnrType']
cat vars []
cat var dummies, []
num vars ['LotArea', 'LotFrontage', 'LotShape']
cat vars ['LotConfig']
cat var dummies, ['LotConfig_Corner', 'LotConfig_CulDSac', 'LotConfig_FR2', 'LotConfig_FR3', 'LotConfig_Inside']
num vars ['PoolQC']
cat vars []
cat var dummies, []
num vars ['GarageArea', 'GarageCars', 'GarageQual', 'GarageYrBlt']
cat vars ['GarageFinish', 'GarageType']
cat var dummies, ['GarageFinish_Fin', 'GarageFinish_No', 'GarageFinish_RFn', 'GarageFinish_Unf', 'GarageType_2Types', 'GarageType_Attchd', 'GarageType_Basment', 'GarageType_BuiltIn', 'GarageType_CarPort', 'GarageType_Detchd', 'GarageType_No']
num vars ['AllPorchSF']
cat vars []
cat var dummies, []
num vars ['WoodDeckSF']
cat vars []
cat var dummies, []
num vars ['KitchenAbvGr', 'KitchenQual']
cat vars []
cat var dummies, []


In [598]:
train_clean.KitchenQual.value_counts()

1    1563
2    1356
Name: KitchenQual, dtype: int64

In [599]:
for s in ['Bsmt','LotFrontage','Porch','Deck']:
    cols = [col for col in train_num.columns if s in col]
    train_num[s+'_absent']=0
    train_num.loc[train_num[cols].sum(axis=1)==0,s+'_absent']=1

In [600]:
print("Number of numerical variables: {}".format(len(train_num.columns)))
print("Number of categorical dummies: {}".format(len(train_cat_dum.columns)))
print("Number of dummies for 0s: {}".format(len([col for col in train_num.columns if 'absent' in col])))
# Join categorical and numerical features
train_small=pd.concat([target_var,train_num, train_cat_dum], axis = 1)
# data clean is done. Wrap the above into a .py file

Number of numerical variables: 45
Number of categorical dummies: 162
Number of dummies for 0s: 4


In [601]:
# check pair-wise high correlations
import copy 
train_clean=copy.deepcopy(train_num)
corr_mat=train_clean.corr()
np.fill_diagonal(corr_mat.values, 0)
for i, col in enumerate(train_clean.columns.values):
    if corr_mat[[col]].abs().sort_values(col,ascending=False).iloc[0,0]>0.9:
        print(corr_mat[[col]].abs().sort_values(col,ascending=False).iloc[:2,:])
        print(train_clean[col].value_counts()[:5])

In [602]:
# explore 0 value for numerical variables
# cols=train_num.columns[train_num.apply(lambda x: x==0).sum()>len(target_var)*0.1]
# fig, axes = plt.subplots(len(cols),1, figsize=(5,25))
# for i in range(len(cols)):
#     print(cols[i])
#     train_clean[[cols[i]]].hist(ax=axes[i])

In [603]:
train_clean.describe()

,BedroomAbvGr,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,BsmtQual,BsmtUnfSF,ExterQual,Fireplaces,Functional,...,YearBuilt,YearRemodAdd,BsmtTotalBath,TotalBath,BsmtFinSF,AllPorchSF,Bsmt_absent,LotFrontage_absent,Porch_absent,Deck_absent
count,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.00000,2919.000000,2919.000000,2919.000000,2919.000000,...,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000
mean,2.860226,1.013703,0.651936,1.502912,1.025351,1.47482,560.579993,1.239294,0.597122,3.920521,...,36.479959,23.528263,0.460260,1.758136,490.837273,89.249743,0.027064,0.166495,0.299075,0.521754
std,0.822693,0.264051,1.039713,0.551565,0.282044,0.55224,439.590889,0.277128,0.646129,0.310630,...,30.336182,20.892061,0.520778,0.642680,477.209821,107.902805,0.162298,0.372589,0.457931,0.499612
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,1.000000,0.000000,1.000000,...,-1.000000,-2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,1.000000,0.000000,1.000000,1.000000,1.00000,220.000000,1.000000,0.000000,4.000000,...,7.000000,4.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.000000,1.000000,0.000000,2.000000,1.000000,2.00000,467.000000,1.000000,1.000000,4.000000,...,35.000000,15.000000,0.000000,2.000000,457.000000,50.000000,0.000000,0.000000,0.000000,1.000000
75%,3.000000,1.000000,1.000000,2.000000,1.000000,2.00000,805.000000,1.500000,1.000000,4.000000,...,54.500000,43.000000,1.000000,2.500000,805.500000,136.000000,0.000000,0.000000,1.000000,1.000000
max,8.000000,2.000000,3.000000,2.000000,2.000000,2.00000,2336.000000,2.000000,4.000000,4.000000,...,136.000000,60.000000,3.000000,5.000000,5644.000000,1207.000000,1.000000,1.000000,1.000000,1.000000


In [604]:
# set the dummy associated with  duplicated dummies;
df_agg = pd.concat([target_var, train_cat.loc[train.Id<=idsTotal,:]], axis = 1).sort_values('SalePrice')
drop_list=[]
for i, col in enumerate(df_agg.columns[1:]):
    s=df_agg.iloc[round(len(df_agg.SalePrice)/2),i+1]
    drop_list.append(col+'_'+str(s))
print(drop_list)

['Alley_2', 'BldgType_2fmCon', 'CentralAir_N', 'Condition1_Norm', 'Condition2_Norm', 'Electrical_SBrkr', 'Exterior1st_Wd Sdng', 'Fence_No', 'Foundation_BrkTil', 'GarageFinish_Unf', 'GarageType_Detchd', 'Heating_GasA', 'HouseStyle_1.5Fin', 'LandContour_Lvl', 'LotConfig_Inside', 'MSZoning_RM', 'MiscFeature_No', 'MoSold_Apr', 'Neighborhood_OldTown', 'RoofMatl_CompShg', 'RoofStyle_Gable', 'SaleType_WD']


In [605]:
train_clean=train_small.drop(drop_list,axis=1)

In [606]:
# check pair-wise high correlations
import copy 
corr_mat=train_clean.corr()
np.fill_diagonal(corr_mat.values, 0)
for i, col in enumerate(train_clean.columns.values):
    if corr_mat[[col]].abs().sort_values(col,ascending=False).iloc[0,0]>0.9:
        print(corr_mat[[col]].abs().sort_values(col,ascending=False).iloc[:2,:])

                 GarageQual
GarageFinish_No    0.952651
GarageType_No      0.946298
              SaleCondition
SaleType_New       0.986573
SalePrice          0.352060
               GarageFinish_No
GarageType_No         0.993331
GarageQual            0.952651
                 GarageType_No
GarageFinish_No       0.993331
GarageQual            0.946298
               SaleType_New
SaleCondition      0.986573
SalePrice          0.357509


In [608]:
train_clean.drop(['GarageFinish_No','SaleType_New'],axis=1,inplace=True) # highly similar to SaleCondition

In [609]:
# check high R2 variables; all have been addressed by the feature engineering above
from sklearn import linear_model
ols = linear_model.LinearRegression()
df=train_clean.drop(['SalePrice'],axis=1)
for i, col in enumerate(df.columns.values):
    x=df.iloc[:,df.columns!=col]
    y=df.iloc[:,i]
    ols.fit(x, y)   #### X needs to be 2-D array; X.reshape(-1,1) to shape 1-D array; x = df[["col1"]]
    if ols.score(x, y)>0.95:
        print(col+' R^2: %.2f' % ols.score(x, y)) ## score: the R^2 of the fitted model

In [531]:
# # check high R2 variables
# import statsmodels.api as sm
# col='MiscFeature_Gar2'
# X=train_clean.iloc[:,train_clean.columns!=col]
# Y=train_clean[col]
# X_add_const = sm.add_constant(X) #now you need to add a constant to X
# ols = sm.OLS(Y, X_add_const)
# ans = ols.fit()
# print(ans.summary())

# # check pair-wise high correlations
# train_clean=copy.deepcopy(train_test)
# corr_mat=train_clean.corr()
# np.fill_diagonal(corr_mat.values, 0)
# for i, col in enumerate(train_clean.columns.values):
#     if corr_mat[[col]].abs().sort_values(col,ascending=False).iloc[0,0]>0.9:
#         print(col,corr_mat[[col]].abs().sort_values(col,ascending=False).iloc[:2,:])

## Cleaner output dataset

In [610]:
# generate dummy variables for categorical variables and numeric variables=0;
print("Number of variables: {}".format(len(train_clean.columns)))
# Join categorical and numerical features
train_clean.to_csv('clean_dataset.csv',index=False)

Number of variables: 184


In [612]:
np.shape(train_clean.T.drop_duplicates().T) #verify no duplicated columns

(2919, 184)

## Model
### Q1. More or fewer features?
### Q2. Lasso vs. Boosted Regression Trees
### Q3. Outlier matters?
### Q4. Even simpler models?
### Q5. Refleciton on the specification for the target variable, log or not

In [759]:
# full variable version
train = pd.read_csv('final_dataset.csv')

In [845]:
# small set version
train = pd.read_csv('clean_dataset.csv')

In [846]:
# test_final set
test_final=train.loc[train.Id>idsTotal,:].drop(['SalePrice','Id'],axis=1)
train=train.loc[train.Id<=idsTotal,:]
y=train.SalePrice #log or not log; choice 1
train=train.drop(['SalePrice','Id'],axis=1)